# Import

In [1]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib widget

# Load and preprocess data

In [2]:
# load data
with open("AllData.pkl","rb") as f:
    data = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'AllData.pkl'

In [ ]:
# create training and validation set

train_airfoils = [5, 13, 21, 25]
validation_airfoils = [9]
test_airfoils = [17]
# dataset shapes:
# x_train [samples, [x_coords,y_coords]]
# x_train_phys [samples, alpha & cl_approx_exp & cl_approx_exp ]
# y_train [samples, cl & cd]


def process_data(airfoils):
    x = []
    x_phys_ext = []
    x_phys = []
    y = []
    for airfoil in airfoils:
        for i in range(len(data[airfoil]["alpha"])):
            x.append(np.reshape(data[airfoil]["coords_reduced"],-1))
            x_phys_ext.append([data[airfoil]["alpha"][i],data[airfoil]["Cl_approx_exp"][i],data[airfoil]["Cd_approx_exp"][i]])
            x_phys.append([data[airfoil]["alpha"][i],data[airfoil]["Cl_approx"][i],data[airfoil]["Cd_approx"][i]])
            y.append([data[airfoil]["Cl_target"][i], data[airfoil]["Cd_target"][i]])
    x = np.array(x )
    x_phys_ext = np.array(x_phys_ext)
    x_phys = np.array(x_phys)
    y = np.array(y)
    return x, x_phys_ext, x_phys, y
    
    
x_train, x_train_phys_ext, x_train_phys, y_train = process_data(train_airfoils)
x_val, x_val_phys_ext, x_val_phys, y_val = process_data(validation_airfoils)
x_test, x_test_phys_ext, x_test_phys, y_test = process_data(test_airfoils)


# Define functions for DNN and PGNN

In [184]:
# function for trtaining single nn
def train_nn(mode,
            x_train,
            x_train_phys,
            y_train,
            x_val,
            x_val_phys,
            y_val,
             verbose
            ):
    # build NN
    inputs = tf.keras.layers.Input(40,)
    if "PGNN" in mode:
        phys = tf.keras.layers.Input(3,)
        inputs2 = inputs
    elif mode == "DDNN":
        phys = tf.keras.layers.Input(1,)
        inputs2 = tf.keras.layers.Concatenate()([inputs,phys])
        
    hidden_nodes = 20
    
    
    if mode == "PGNN_1":
        merged = tf.keras.layers.Concatenate()([inputs2,phys])
        h1 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h1 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(inputs2)
    
    if mode == "PGNN_2":
        merged = tf.keras.layers.Concatenate()([h1,phys])
        h2 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h2 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h1)
    
    if mode == "PGNN_3":
        merged = tf.keras.layers.Concatenate()([h2,phys])
        h3 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h3 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h2)
        
        
    if mode == "PGNN_4":
        merged = tf.keras.layers.Concatenate()([h3,phys])
        h4 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h4 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h3)
        
    if mode == "PGNN_5":
        merged = tf.keras.layers.Concatenate()([h4,phys])
        h5 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h5 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h4)    
    
    if mode == "PGNN_6":
        merged = tf.keras.layers.Concatenate()([h5,phys])
        h6 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(merged)
    else:
        h6 = tf.keras.layers.Dense(hidden_nodes, activation=tf.nn.relu, 
                                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                   bias_initializer=tf.keras.initializers.Zeros()
                                  )(h5)  
        
    outputs = tf.keras.layers.Dense(2, activation=None, 
                               kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                               bias_initializer=tf.keras.initializers.Zeros()
                              )(h6)
    model = tf.keras.Model(inputs=[inputs,phys],outputs = outputs)
   
    # compiel NN
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss = "mse",
    )
        

        
        
    history = model.fit(
        x = [x_train,x_train_phys],
        y = y_train,
        batch_size = 64,
        epochs = 200,
        callbacks = [
                tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    min_delta=0,
                    patience=8,
                    verbose=0,
                    restore_best_weights=True,
                    start_from_epoch=0,
                    ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor="val_loss",
                    factor=0.1,
                    patience=3,
                    min_delta=0,
                    )
                ],
        validation_data = ([x_val, x_val_phys],y_val),
            verbose = verbose
        )
    return model, history

In [185]:
# function for creating an ensemble

def build_ensemble(
            nns, 
            mode, 
            x_train,
            x_train_phys,
            y_train,
            x_val,
            x_val_phys,
            y_val,
            verbose = 1
            ):
    models, histories = [], []
    for i in range(nns):
        model, history = train_nn(mode, x_train,x_train_phys, y_train,x_val,x_val_phys,y_val,verbose-1)
        models.append(model)
        histories.append(history)
        if verbose>0:
            print("NN ",i, 
                  ", Epochs",len(history.history["loss"]), 
                  ", loss", history.history["loss"][-1], 
                  ", val_loss", history.history["val_loss"][-1])
    
    return models, histories

# Create Ensembles

In [187]:
nns = 10


pgnns, pgnn_hists = build_ensemble(            
            nns = nns,
            mode = "PGNN_3", 
            x_train = x_train,
            x_train_phys = x_train_phys,
            y_train = y_train,
            x_val = x_val,
            x_val_phys = x_val_phys,
            y_val = y_val,
            verbose = 1)

pgnns_exp, pgnn_exp_hists = build_ensemble(            
            nns =nns,
            mode = "PGNN_4", 
            x_train = x_train,
            x_train_phys = x_train_phys_ext,
            y_train = y_train,
            x_val = x_val,
            x_val_phys = x_val_phys_ext,
            y_val = y_val,
            verbose = 1)

dnn, dnn_hists = build_ensemble(            
            nns = nns,
            mode = "DDNN", 
            x_train = x_train,
            x_train_phys = x_train_phys[:,0],
            y_train = y_train,
            x_val = x_val,
            x_val_phys = x_val_phys[:,0],
            y_val = y_val,
            verbose = 1)

NN  0 , Epochs 9 , loss 0.5556554794311523 , val_loss 0.6682834029197693
NN  1 , Epochs 44 , loss 0.02065730094909668 , val_loss 0.025566376745700836
NN  2 , Epochs 78 , loss 0.013449029996991158 , val_loss 0.018490538001060486
NN  3 , Epochs 118 , loss 0.026968831196427345 , val_loss 0.034359339624643326
NN  4 , Epochs 46 , loss 0.0111951669678092 , val_loss 0.01874694414436817
NN  5 , Epochs 47 , loss 0.019409343600273132 , val_loss 0.028651461005210876
NN  6 , Epochs 15 , loss 0.2762572467327118 , val_loss 0.2985233664512634
NN  7 , Epochs 54 , loss 0.013658598996698856 , val_loss 0.022636888548731804
NN  8 , Epochs 189 , loss 0.05055311322212219 , val_loss 0.05028071999549866
NN  9 , Epochs 43 , loss 0.016410792246460915 , val_loss 0.023076463490724564
NN  0 , Epochs 41 , loss 0.010767693631350994 , val_loss 0.013793961144983768
NN  1 , Epochs 32 , loss 0.014669984579086304 , val_loss 0.02002045139670372
NN  2 , Epochs 34 , loss 0.0178357120603323 , val_loss 0.02214459888637066
NN 

In [188]:
# test accuracy

def MSE(x,y):
    x = np.reshape(np.array(x),-1)
    y = np.reshape(np.array(y),-1)
    z = np.mean((x-y)**2)
    return z

mean_pred=[]
mse_5 = []
for model in pgnns:
    y_pred = model.predict( 
        x = [x_test,x_test_phys],verbose=0
    )
    #mse_5.append((MSE( y_pred[:50], y_test[:50]) + MSE(y_pred[150:], y_test[150:]))/2)
    mse_5.append(MSE( y_pred, y_test))
    mean_pred.append(y_pred)

mean_pred_ext = []
mse_ext_5=[]
for model in pgnns_exp:
    y_pred = model.predict( 
        x = [x_test,x_test_phys_ext],verbose=0
    )
    #mse_ext_5.append((MSE( y_pred[:50], y_test[:50]) + MSE(y_pred[150:], y_test[150:]))/2)
    mse_ext_5.append(MSE( y_pred, y_test) )
    mean_pred_ext.append(y_pred)

mean_pred_dnn = []
mse_dnn_5 = []
for model in dnn:
    y_pred = model.predict( 
        x = [x_test,x_test_phys[:,0]], verbose=0
    )
    #mse_dnn_5.append((MSE( y_pred[:50], y_test[:50]) + MSE(y_pred[150:], y_test[150:]))/2)
    mse_dnn_5.append(MSE( y_pred, y_test))
    mean_pred_dnn.append(y_pred)
mean_pred_5 = np.array(mean_pred).mean(axis=0)
mean_pred_ext_5 = np.array(mean_pred_ext).mean(axis=0)
mean_pred_dnn_5 = np.array(mean_pred_dnn).mean(axis=0)


In [ ]:
# Repeat with training airfoil 5 (no simplified PBM input) replaced by training airfoil 6

In [189]:
# create alternative training set

train_airfoils = [6, 13, 21, 25]
validation_airfoils = [9]
test_airfoils = [17]
# dataset shapes:
# x_train [samples, [x_coords,y_coords]]
# x_train_phys [samples, alpha & cl_approx_exp & cl_approx_exp ]
# y_train [samples, cl & cd]


def process_data(airfoils):
    x = []
    x_phys_ext = []
    x_phys = []
    y = []
    for airfoil in airfoils:
        for i in range(len(data[airfoil]["alpha"])):
            x.append(np.reshape(data[airfoil]["coords_reduced"],-1))
            x_phys_ext.append([data[airfoil]["alpha"][i],data[airfoil]["Cl_approx_exp"][i],data[airfoil]["Cd_approx_exp"][i]])
            x_phys.append([data[airfoil]["alpha"][i],data[airfoil]["Cl_approx"][i],data[airfoil]["Cd_approx"][i]])
            y.append([data[airfoil]["Cl_target"][i], data[airfoil]["Cd_target"][i]])
    x = np.array(x )
    x_phys_ext = np.array(x_phys_ext)
    x_phys = np.array(x_phys)
    y = np.array(y)
    return x, x_phys_ext, x_phys, y
    
    
x_train, x_train_phys_ext, x_train_phys, y_train = process_data(train_airfoils)
x_val, x_val_phys_ext, x_val_phys, y_val = process_data(validation_airfoils)
x_test, x_test_phys_ext, x_test_phys, y_test = process_data(test_airfoils)


In [190]:
# create pgnn ensemble with extrapolated_data

nns = 10

if 1:
    pgnns, pgnn_hists = build_ensemble(            
            nns = nns,
            mode = "PGNN_3", 
            x_train = x_train,
            x_train_phys = x_train_phys,
            y_train = y_train,
            x_val = x_val,
            x_val_phys = x_val_phys,
            y_val = y_val,
            verbose = 1)

if 1:
    pgnns_exp, pgnn_exp_hists = build_ensemble(            
            nns =nns,
            mode = "PGNN_4", 
            x_train = x_train,
            x_train_phys = x_train_phys_ext,
            y_train = y_train,
            x_val = x_val,
            x_val_phys = x_val_phys_ext,
            y_val = y_val,
            verbose = 1)

dnn, dnn_hists = build_ensemble(            
            nns = nns,
            mode = "DDNN", 
            x_train = x_train,
            x_train_phys = x_train_phys[:,0],
            y_train = y_train,
            x_val = x_val,
            x_val_phys = x_val_phys[:,0],
            y_val = y_val,
            verbose = 1)

NN  0 , Epochs 48 , loss 0.015738071873784065 , val_loss 0.01792730763554573
NN  1 , Epochs 73 , loss 0.008283112198114395 , val_loss 0.01176405418664217
NN  2 , Epochs 75 , loss 0.013804882764816284 , val_loss 0.015070997178554535
NN  3 , Epochs 45 , loss 0.0230936910957098 , val_loss 0.025332069024443626
NN  4 , Epochs 53 , loss 0.012182355858385563 , val_loss 0.01656709983944893
NN  5 , Epochs 38 , loss 0.012372604571282864 , val_loss 0.016915082931518555
NN  6 , Epochs 59 , loss 0.009333360940217972 , val_loss 0.013497688807547092
NN  7 , Epochs 56 , loss 0.008204239420592785 , val_loss 0.012401836924254894
NN  8 , Epochs 43 , loss 0.01958627998828888 , val_loss 0.022185098379850388
NN  9 , Epochs 100 , loss 0.019025394693017006 , val_loss 0.02218046598136425
NN  0 , Epochs 66 , loss 0.0056801591999828815 , val_loss 0.006791104096919298
NN  1 , Epochs 40 , loss 0.0063677276484668255 , val_loss 0.007712156046181917
NN  2 , Epochs 30 , loss 0.003738269442692399 , val_loss 0.003897626

In [191]:
# test accuracy

def MSE(x,y):
    x = np.reshape(np.array(x),-1)
    y = np.reshape(np.array(y),-1)
    z = np.mean((x-y)**2)
    return z

mean_pred=[]
mse_6 = []
for model in pgnns:
    y_pred = model.predict( 
        x = [x_test,x_test_phys],verbose=0
    )
    
    #mse_6.append((MSE( y_pred[:50], y_test[:50]) + MSE(y_pred[150:], y_test[150:]))/2)
    mse_6.append(MSE( y_pred, y_test))
    mean_pred.append(y_pred)

mean_pred_ext = []
mse_ext_6=[]
for model in pgnns_exp:
    y_pred = model.predict( 
        x = [x_test,x_test_phys_ext],verbose=0
    )
    
    #mse_ext_6.append((MSE( y_pred[:50], y_test[:50]) + MSE(y_pred[150:], y_test[150:]))/2)
    mse_ext_6.append(MSE( y_pred, y_test))
    mean_pred_ext.append(y_pred)

mean_pred_dnn = []
mse_dnn_6 = []
for model in dnn:
    y_pred = model.predict( 
        x = [x_test,x_test_phys[:,0]], verbose=0
    )
    #mse_dnn_6.append((MSE( y_pred[:50], y_test[:50]) + MSE(y_pred[150:], y_test[150:]))/2)
    mse_dnn_6.append(MSE( y_pred, y_test))
    mean_pred_dnn.append(y_pred)
    

mean_pred_xfoil = np.array(MSE( x_test_phys[:,1:], y_test))
mean_pred_xfoil_ext = np.array(MSE( x_test_phys_ext[:,1:], y_test))
    
mean_pred_6 = np.array(mean_pred).mean(axis=0)
mean_pred_ext_6 = np.array(mean_pred_ext).mean(axis=0)
mean_pred_dnn_6 = np.array(mean_pred_dnn).mean(axis=0)


In [192]:
# save plot data

mse = {
    "mse_5":mse_5,
    "mse_6":mse_6,
    "mse_ext_5":mse_ext_5,
    "mse_ext_6":mse_ext_6,
    "mse_dnn_5":mse_dnn_5,
    "mse_dnn_6":mse_dnn_6,
    "mse_xfoil":mean_pred_xfoil,
    "mse_xfoil_ext":mean_pred_xfoil_ext,
}

In [88]:
mse["mse_xfoil"]=mean_pred_xfoil
mse["mse_xfoil_ext"]=mean_pred_xfoil_ext
#with open("Plot_data/purified.pkl", "wb") as f:
#    pickle.dump(mse,f)


# Plot results

In [4]:
with open("Plot_data/purified.pkl", "rb") as f:
    mse = pickle.load(f)
    
import matplotlib
font = {
        'size'   : 15}

matplotlib.rc('font', **font)

fig, ax = plt.subplots(figsize=(7,6))
plt.yscale("log")
plt.boxplot(mse["mse_5"],positions = [0], labels=["Zero t5"])
plt.boxplot(mse["mse_6"],positions = [0.4], labels=["Zero t6"])
plt.boxplot(mse["mse_ext_5"],positions = [1.2], labels=["Viterna t5"])
plt.boxplot(mse["mse_ext_6"],positions = [1.6], labels=["Viterna t6"])
plt.boxplot(mse["mse_dnn_5"],positions = [2.4], labels=["DNN t5"])
plt.boxplot(mse["mse_dnn_6"],positions = [2.8], labels=["DNN t6"])

plt.plot([-0.2,3],[mse["mse_xfoil"],mse["mse_xfoil"]],label="Simplified PBM", color="#1f77b4")
plt.plot([-0.2,3],[mse["mse_xfoil_ext"],mse["mse_xfoil_ext"]],label="Simplified PBM with extrapolation", color="#1f77b4", linestyle = "--")
plt.legend(prop={'size':14}, loc = "upper left", bbox_to_anchor=(0,0.9))
#plt.legend(prop={'size':14}, loc = "upper left", bbox_to_anchor=(0,1))
#plt.ylim(0.0018,0.99)
#plt.ylim(0.0015,1.2)
#plt.xlim(-0.2,3)

labels = ["A5", "A6", "A5", "A6", "A5", "A6"]
group = ["\nPGNN with\nsimplified\nPBM", "\nPGNN with\nsimplified PBM\nand extrapolation", "\nDDNN"]
ticks= ax.get_xticks()
ax.set_xticks(ticks,  minor=True)
ax.set_xticklabels(labels, minor=True)
ax.set_xticks([0.2,1.4,2.6], minor=False)
ax.set_xticklabels(group, minor=False)
plt.ylabel("Mean squared error", size=16)
plt.xlabel("\nModel configuration", size=16)
plt.grid(axis="y")
plt.tick_params(axis='x', which='major', bottom=False)
plt.tight_layout()
plt.savefig("purified.png")

FileNotFoundError: [Errno 2] No such file or directory: 'Plot_data/purified.pkl'

# Test which injection layer is best

In [17]:
def MSE(x,y):
    x = np.reshape(np.array(x),-1)
    y = np.reshape(np.array(y),-1)
    z = np.mean((x-y)**2)
    return z

In [32]:
# function for creating an ensemble
nns = 10
mmodels = []
hiists=[]

for i in range(1,7):
    models, hists = build_ensemble(
                nns, 
                mode = "PGNN_"+str(i), 
                x_train=x_train,
                x_train_phys=x_train_phys_ext,
                y_train=y_train,
                x_val=x_val,
                x_val_phys = x_val_phys_ext,
                y_val = y_val,
                verbose = 1
    )
    mmodels.append(models)
    hiists.append(hists)

NN  0 , Epochs 146 , loss 0.014099065214395523 , val_loss 0.013796405866742134
NN  1 , Epochs 32 , loss 0.051910728216171265 , val_loss 0.05133821442723274
NN  2 , Epochs 56 , loss 0.012860295362770557 , val_loss 0.017096925526857376
NN  3 , Epochs 29 , loss 0.3254958391189575 , val_loss 0.35588863492012024
NN  4 , Epochs 40 , loss 0.06807991862297058 , val_loss 0.0647411122918129
NN  5 , Epochs 41 , loss 0.05170825123786926 , val_loss 0.05093712732195854
NN  6 , Epochs 93 , loss 0.03945550322532654 , val_loss 0.03646839037537575
NN  7 , Epochs 52 , loss 0.04248170927166939 , val_loss 0.04519762843847275
NN  8 , Epochs 35 , loss 0.06062361225485802 , val_loss 0.06689499318599701
NN  9 , Epochs 42 , loss 0.061278052628040314 , val_loss 0.06168333441019058
NN  0 , Epochs 56 , loss 0.047491248697042465 , val_loss 0.048520464450120926
NN  1 , Epochs 30 , loss 0.1282922774553299 , val_loss 0.11211810261011124
NN  2 , Epochs 52 , loss 0.008868257515132427 , val_loss 0.009737835265696049
NN  

In [51]:
    
mse = []

for i in range(len(mmodels)):
    mse_2=[]
    for model in mmodels[i]:
        y_pred = model.predict( 
            x = [x_test,x_test_phys_ext],verbose=0
        )

        #mse_ext_6.append((MSE( y_pred[:50], y_test[:50]) + MSE(y_pred[150:], y_test[150:]))/2)
        mse_2.append(MSE( y_pred, y_test))
    mse.append(mse_2)
with open("Plot_data/injection_layer_extended.pkl", "wb") as f:
    pickle.dump(mse,f)

In [54]:
with open("Plot_data/injection_layer_extended.pkl", "rb") as f:
    mse = pickle.load(f)
    
    
import matplotlib
font = {
        'size'   : 16}

matplotlib.rc('font', **font)
plt.figure()
plt.yscale("log")
for i in range(len(mse)):
    plt.boxplot(mse[i],positions=[i])
plt.ylabel("Mean squared error")
plt.xticks([0,1,2,3,4,5], ["L1","L2","L3","L4","L5","L6"])
plt.xlabel("Injection layer")
plt.tight_layout()
plt.savefig("injection_layer_extended.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …